In [7]:
import timm

model = timm.create_model('vit_small_resnet26d_224')
print('the number of model parameters: {:,}'.format(sum([p.data.nelement() for p in model.parameters()])))

the number of model parameters: 63,608,392


In [6]:
import torch
from torchvision import models

from vit_pytorch.distill import DistillableViT, DistillWrapper

teacher = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

model = DistillableViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 100,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)
# print()
print('the number of model parameters: {:,}'.format(sum([p.data.nelement() for p in model.parameters()])))

distiller = DistillWrapper(
    student = model,
    teacher = teacher,
    temperature = 3,           # temperature of distillation
    alpha = 0.5,               # trade between main loss and distillation loss
    hard = False               # whether to use soft or hard distillation
)

img = torch.randn(2, 3, 256, 256)
labels = torch.randint(0, 100, (2,))
print(img.shape)
print(labels.shape)

loss = distiller(img, labels)
loss.backward()

# after lots of training above ...

pred = model(img) # (2, 1000)
print(pred.shape)


the number of model parameters: 41,116,772
torch.Size([2, 3, 256, 256])
torch.Size([2])


RuntimeError: The size of tensor a (1000) must match the size of tensor b (100) at non-singleton dimension 1